## Алиса или нет?

**Есть трэйн выборка посещения людьми разных сайтов (до 10) и один из них идентифицирован как Алиса!  
Надо обучиться так, чтобы в тестовых выборках тоже определять Алису!**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import datetime
from sklearn.preprocessing import StandardScaler

In [7]:
train = pd.read_csv("train_sessions.csv", index_col='session_id')
train.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   site1   253561 non-null  int64  
 1   time1   253561 non-null  object 
 2   site2   250098 non-null  float64
 3   time2   250098 non-null  object 
 4   site3   246919 non-null  float64
 5   time3   246919 non-null  object 
 6   site4   244321 non-null  float64
 7   time4   244321 non-null  object 
 8   site5   241829 non-null  float64
 9   time5   241829 non-null  object 
 10  site6   239495 non-null  float64
 11  time6   239495 non-null  object 
 12  site7   237297 non-null  float64
 13  time7   237297 non-null  object 
 14  site8   235224 non-null  float64
 15  time8   235224 non-null  object 
 16  site9   233084 non-null  float64
 17  time9   233084 non-null  object 
 18  site10  231052 non-null  float64
 19  time10  231052 non-null  object 
 20  target  253561 non-null  int64  
dtypes: float64

In [12]:
train[train.target == 1]['time1']

session_id
251175    2013-02-12 16:25:10
196388    2013-02-12 16:32:27
172448    2013-02-12 16:32:53
70129     2013-02-12 16:33:50
167235    2013-02-12 16:33:55
                 ...         
188473    2014-04-15 17:42:12
40139     2014-04-15 17:42:36
205622    2014-04-15 17:42:49
60720     2014-04-15 18:06:02
244233    2014-04-15 18:07:48
Name: time1, Length: 2297, dtype: object

### Получение колонки дней недели

In [35]:
train[train.target == 1]['time1'].sort_values().iloc[0]

Timestamp('2013-02-12 16:25:10')

In [34]:
train[train.target == 1]['time1'].sort_values().iloc[0].date()

datetime.date(2013, 2, 12)

In [33]:
train[train.target == 1]['time1'].sort_values().iloc[0].date().isoweekday()

2

In [38]:
day_week = train['time1'].apply(lambda time1: time1.date().isoweekday()) # Получили все дни недели у всех
day_week.head()

session_id
1    4
2    6
3    1
4    5
5    5
Name: time1, dtype: int64

In [47]:
day_week.value_counts()

3    55971
2    48659
4    44147
5    41140
1    40513
6    15799
7     7332
Name: time1, dtype: int64

### Другие вычисления

In [4]:
train['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

In [130]:
train.shape[0] - train.dropna().shape[0]

22509

In [131]:
round(train['target'].value_counts()[1] / train['target'].value_counts()[0] * 100, 2)

0.91

In [132]:
train[train.target == 1]['time1'].sort_values().head(1)

session_id
251175    2013-02-12 16:25:10
Name: time1, dtype: object

In [5]:
test = pd.read_csv("test_sessions.csv", index_col='session_id');

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   site1   82797 non-null  int64  
 1   time1   82797 non-null  object 
 2   site2   81308 non-null  float64
 3   time2   81308 non-null  object 
 4   site3   80075 non-null  float64
 5   time3   80075 non-null  object 
 6   site4   79182 non-null  float64
 7   time4   79182 non-null  object 
 8   site5   78341 non-null  float64
 9   time5   78341 non-null  object 
 10  site6   77566 non-null  float64
 11  time6   77566 non-null  object 
 12  site7   76840 non-null  float64
 13  time7   76840 non-null  object 
 14  site8   76151 non-null  float64
 15  time8   76151 non-null  object 
 16  site9   75484 non-null  float64
 17  time9   75484 non-null  object 
 18  site10  74806 non-null  float64
 19  time10  74806 non-null  object 
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


## **Колонки со временем приведем ко времени**

In [13]:
times = ['time%s' % i for i in range(1, 11)] #Прикольная штука, быстро подставить индекс в название
times

['time1',
 'time2',
 'time3',
 'time4',
 'time5',
 'time6',
 'time7',
 'time8',
 'time9',
 'time10']

In [14]:
train[times] = train[times].apply(pd.to_datetime) # Можно посмотреть типы данных через инфо До и После

In [15]:
test[times] = test[times].apply(pd.to_datetime) # И в тестовой выборке тоже приводим к временному формату данных

In [137]:
train.sort_values(by='time1', inplace=True) # Отсортируем по времени тайм1 (только зачем?)

In [138]:
train.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [139]:
sites = ['site%s' % i for i in range(1, 11)]          # Также быстро делаем список с сайтами

In [140]:
train[sites] = train[sites].fillna(0).astype("int")   # Меняем Nan на нули и делаем их интами в трэйне

In [141]:
test[sites] = test[sites].fillna(0).astype("int")     #...и в тесте

**Можно подгрузить словарик сайтов (он в удобной форме pickle), сделать из него DF**

In [142]:
with open("site_dic.pkl", "rb") as inp_file:
    site_dict = pickle.load(inp_file)        # Это сделает словарь Сайт : номер у нас в таблице
site_dict_df = pd.DataFrame(list(site_dict.keys()),
                            index=list(site_dict.values()),
                            columns=['site'])
site_dict_df.head()

,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


## Интересная предобработка - разряженые матрицы

**Чтобы применить Bag of words для всех элементов, объединим выборки трэйн и тест**  
Предполагаем, что Алиса ходила на какие-то определенные сайты, поэтому делаем из каждого сайта бинарный признак, соответственно 1-посещал, 0-не посещал.  
Но тогда у нас будет большое кол-во столбцов с сайтами и единички там, где сайты посещались. Т.е. у каждого человека (в каждой строчке) будет куча нулей (сайтов **50000**, посетил 10 и менее) и только 10 или меньше единиц. Матрица - разряженая. Надо применить другое представление

In [143]:
# Предварительно сделаем y_train
y_train = train['target']

In [144]:
indx_max = train.shape[0] # Запомним какая строчка в трэйне последняя
indx_max

253561

In [145]:
full_df = pd.concat([train.drop("target", axis=1), test]) # Объединим без целевой переменной, не забываем []

Теперь посмотрим, что можно выжать только из колонок с сайтами, без времени

In [146]:
full_sites = full_df[sites]
full_sites.head()            #Только сайты!

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


In [147]:
full_sites.shape # Это так, для сравнения с train

(336358, 10)

In [148]:
full_sites_flatten = full_sites.values.flatten() # Требует дополнительного изучения!

In [149]:
full_sites_flatten.shape

(3363580,)

In [150]:
full_sites_sparse = csr_matrix(([1] * full_sites_flatten.shape[0], full_sites_flatten,
                              range(0, full_sites_flatten.shape[0] + 10, 10)))[:, 1:]

In [151]:
# from scipy.sparse import csr_matrix

csr_matrix(([1] * full_sites_flatten.shape[0], full_sites_flatten, range(0, full_sites_flatten.shape[0] + 10, 10)))

<336358x48372 sparse matrix of type '<class 'numpy.int64'>'
	with 3363580 stored elements in Compressed Sparse Row format>

In [152]:
full_sites_sparse # Это сжатый тип, без первого столбца, как я понимаю, только данные!

<336358x48371 sparse matrix of type '<class 'numpy.int64'>'
	with 3195430 stored elements in Compressed Sparse Row format>

In [153]:
full_sites_sparse.shape

(336358, 48371)

### Обучаем

**Поделим выборку обратно на трэйн и тест, используем максим индекс строки в трэйне, который запоминали**

In [154]:
X_train_sparse = full_sites_sparse[:indx_max, :]  # y_train у нас есть, мы забрали его из train перед объединением

In [155]:
X_test_sparse = full_sites_sparse[indx_max:, :]

In [156]:
X_train_sparse.shape

(253561, 48371)

In [157]:
y_train.shape # Видим, что совпадает!

(253561,)

In [158]:
# 82797 ответа надо послать на Кёгл!
X_test_sparse.shape

(82797, 48371)

**Данных достаточно много, чтобы не проводить кросс-валидацию**  
Это требует доп проверки, но в данном случае проверка опущена  
  
Напишем простую ф-ю, чтобы по 10 раз не возвращаться к LogReg.  
Внесем в нее основные параметры для LR  
Трэйн выборку тоже будем разбивать на обучающую и проверочную!

In [159]:
from sklearn.linear_model import LogisticRegression

In [160]:
%%time
# Просто для оценки времени работы ф-ии, при сложных DFтак делать не надо, долго придется ждать
l_r = LogisticRegression(max_iter=1000, n_jobs=-1, random_state=17)
l_r_fit = l_r.fit(X_train_sparse, y_train)

CPU times: user 149 ms, sys: 182 ms, total: 332 ms
Wall time: 15.3 s


In [161]:
# Посмотрим что здесь
l_r_fit

LogisticRegression(max_iter=1000, n_jobs=-1, random_state=17)

In [162]:
# Смотрим что выдаст обученная модель на тестовых данных, по одной (любой) из строчек
l_r_fit.predict(X_test_sparse[:20, :])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [163]:
# А можно посмотреть вероятность такого угадывания в конкретной строчке
# В ответе 1стлб - вероятность присвоения класса "0", 2стлб - вероятность присвоения класса "1".
l_r_fit.predict_proba(X_test_sparse[:20, :])

array([[9.97933834e-01, 2.06616568e-03],
       [9.99999996e-01, 4.30938660e-09],
       [9.99999981e-01, 1.85278640e-08],
       [9.99999979e-01, 2.10384069e-08],
       [9.99969858e-01, 3.01419727e-05],
       [9.99820011e-01, 1.79988870e-04],
       [9.99576063e-01, 4.23936965e-04],
       [9.99888808e-01, 1.11191528e-04],
       [9.99277882e-01, 7.22118366e-04],
       [8.96470686e-01, 1.03529314e-01],
       [9.99971190e-01, 2.88096392e-05],
       [9.99913534e-01, 8.64663958e-05],
       [9.99589498e-01, 4.10501602e-04],
       [6.40905948e-01, 3.59094052e-01],
       [9.99955876e-01, 4.41240209e-05],
       [9.96362717e-01, 3.63728317e-03],
       [9.84827459e-01, 1.51725411e-02],
       [9.99968517e-01, 3.14830687e-05],
       [9.99117716e-01, 8.82284351e-04],
       [9.99999747e-01, 2.53225828e-07]])

In [164]:
# Например, еще одна строчка
l_r_fit.predict_proba(X_test_sparse[1, :])

array([[9.99999996e-01, 4.30938660e-09]])

**Но нас будет интересовать вероятность появления класса 1, т.е. вероятность, что это Алиса!**

In [165]:
l_r_fit.predict_proba(X_test_sparse[:20, :])[:, 1]

array([2.06616568e-03, 4.30938660e-09, 1.85278640e-08, 2.10384069e-08,
       3.01419727e-05, 1.79988870e-04, 4.23936965e-04, 1.11191528e-04,
       7.22118366e-04, 1.03529314e-01, 2.88096392e-05, 8.64663958e-05,
       4.10501602e-04, 3.59094052e-01, 4.41240209e-05, 3.63728317e-03,
       1.51725411e-02, 3.14830687e-05, 8.82284351e-04, 2.53225828e-07])

In [166]:
def LogReg_roc_auc1(X, y, C=1.0, ratio=0.9, seed=17, m_iter=100):
    # X, y - матрица фичей, вектор целевой переменной соответственно,
    # С - кэф регуляризации
    # ratio - % в каком отношении делим выборку на трейн и тест
    # seed - random_state, лучше фиксировать везде.
    
    train_len = int(X.shape[0] * ratio) 
                                               # Вроде разбиение такое можно функцией сделать train_test_split
                                               # sklearn.model_selection.train_test_split
    X_train = X[train_len:, :]
    X_valid = X[:train_len, :]
    y_train = y[train_len:]
    y_valid = y[:train_len]
    
    # Создаем объект класса
    log_reg = LogisticRegression(C=C, random_state=seed, n_jobs=-1, max_iter=m_iter)
    
    # Собственно обучение, или тренировка обуч.выборке
    log_reg.fit(X_train, y_train)
    
    # Теперь вычисляем, какую вероятность попадания в 1 будет иметь модель на тестовой выборке
    valid_pred = log_reg.predict_proba(X_valid)[:, 1] #Вот она - вероятность класса 1 или вероятность Алисы
    
    # Выход - Roc-Auc (насколько близко к еденице реальные данные и полученные нами)
    return roc_auc_score(y_valid, valid_pred)

In [167]:
LogReg_roc_auc1(X_train_sparse, y_train)

0.8237843929636158

In [168]:
# Так дает другой результат???????????
# Из-за random_state, что ли или я не правильно выборки делю в верхнем?
def LogReg_roc_auc(X, y, C=0.7, ratio=0.75, seed=17, m_iter=500):
    # X, y - матрица фичей, вектор целевой переменной соответственно,
    # С - кэф регуляризации
    # ratio - % в каком отношении делим выборку на трейн и тест
    # seed - random_state, лучше фиксировать везде.
    
   
    # Вот реализация с функцией разбиения
    # sklern.model_selection.train_test_split
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=ratio, random_state=seed)

    
    # Создаем объект класса
    log_reg = LogisticRegression(C=C, random_state=seed, n_jobs=-1, max_iter=m_iter)
    
    # Собственно обучение, или тренировка обуч.выборке
    log_reg.fit(X_train, y_train)
    
    # Теперь вычисляем, какую вероятность попадания в 1 будет иметь модель на тестовой выборке
    valid_pred = log_reg.predict_proba(X_valid)[:, 1] #Вот она - вероятность класса 1 или вероятность Алисы
    
    # Выход - Roc-Auc (насколько близко к еденице реальные данные и полученные нами)
    return roc_auc_score(y_valid, valid_pred)

In [169]:
# Не хилый результат сходу получился, посмотрим, что можно еще сделать
LogReg_roc_auc(X_train_sparse, y_train)

0.9677402334379981

In [170]:
%%time
log_reg = LogisticRegression(C=0.7, random_state=17, n_jobs=-1, max_iter=500)
log_reg.fit(X_train_sparse, y_train)
test_predict = log_reg.predict_proba(X_test_sparse)[:, 1]

CPU times: user 53.5 ms, sys: 16.3 ms, total: 69.7 ms
Wall time: 12.4 s


In [171]:
test_predict.shape

(82797,)

In [172]:
test_predict[:10], test.index[:10]

(array([2.72527004e-03, 2.35963062e-08, 4.66479936e-08, 9.92477935e-08,
        4.22188984e-05, 2.69950575e-04, 6.01954593e-04, 1.77648133e-04,
        1.02582135e-03, 1.03515796e-01]),
 Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64', name='session_id'))

In [173]:
# Меняем индексы, чтобы первый был 1, а не 0. Зададим имена колонок.
pd.Series(test_predict, index=range(1, test_predict.shape[0] + 1), name="target")

1        2.725270e-03
2        2.359631e-08
3        4.664799e-08
4        9.924779e-08
5        4.221890e-05
             ...     
82793    2.503472e-05
82794    3.197908e-05
82795    9.221669e-03
82796    6.068064e-04
82797    3.141524e-05
Name: target, Length: 82797, dtype: float64

In [174]:
# Переносим в csv файл
pd.Series(test_predict, index=range(1, test_predict.shape[0] + 1), name="target"). \
to_csv("benchmark1.csv", header=True, index_label="session_id")

In [175]:
!head benchmark1.csv

session_id,target
1,0.0027252700390564275
2,2.3596306183785566e-08
3,4.664799356544051e-08
4,9.924779353746127e-08
5,4.2218898403100165e-05
6,0.000269950574977715
7,0.0006019545928958584
8,0.00017764813284062295
9,0.0010258213540001815


In [176]:
test.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35,2014-10-04 11:19:53,22,2014-10-04 11:19:54,321,2014-10-04 11:19:54,23,2014-10-04 11:19:54,2211,2014-10-04 11:19:54,6730,2014-10-04 11:19:54,21,2014-10-04 11:19:54,44582,2014-10-04 11:20:00,15336,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782,2014-07-03 11:00:53,782,2014-07-03 11:00:58,782,2014-07-03 11:01:06,782,2014-07-03 11:01:09,782,2014-07-03 11:01:10,782,2014-07-03 11:01:23,782,2014-07-03 11:01:29,782,2014-07-03 11:01:30,782,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55,2014-12-05 15:55:13,55,2014-12-05 15:55:14,55,2014-12-05 15:56:15,55,2014-12-05 15:56:16,55,2014-12-05 15:56:17,55,2014-12-05 15:56:18,55,2014-12-05 15:56:19,1445,2014-12-05 15:56:33,1445,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022,2014-11-04 10:03:19,50,2014-11-04 10:03:20,222,2014-11-04 10:03:21,202,2014-11-04 10:03:21,3374,2014-11-04 10:03:22,50,2014-11-04 10:03:22,48,2014-11-04 10:03:22,48,2014-11-04 10:03:23,3374,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301,2014-05-16 15:05:32,301,2014-05-16 15:05:33,66,2014-05-16 15:05:39,67,2014-05-16 15:05:40,69,2014-05-16 15:05:40,70,2014-05-16 15:05:40,68,2014-05-16 15:05:40,71,2014-05-16 15:05:40,167,2014-05-16 15:05:44


# Добавляем характеристики вместо времени

In [177]:
train.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0


In [178]:
train[times].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 21669 to 204762
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   time1   253561 non-null  datetime64[ns]
 1   time2   250098 non-null  datetime64[ns]
 2   time3   246919 non-null  datetime64[ns]
 3   time4   244321 non-null  datetime64[ns]
 4   time5   241829 non-null  datetime64[ns]
 5   time6   239495 non-null  datetime64[ns]
 6   time7   237297 non-null  datetime64[ns]
 7   time8   235224 non-null  datetime64[ns]
 8   time9   233084 non-null  datetime64[ns]
 9   time10  231052 non-null  datetime64[ns]
dtypes: datetime64[ns](10)
memory usage: 21.3 MB


In [28]:
# Так легко вытащить год и также месяц из типа datetime64
train["time1"][1].year

2014

In [22]:
# Они еще и интовые!!!!!!!!!!!!!!!!!!!!!!!!!!
type(train["time1"][1].month)

int

In [181]:
# Вот так это работает
train['time1'].apply(lambda time: time.year).head() 

session_id
21669     2013
54843     2013
77292     2013
114021    2013
146670    2013
Name: time1, dtype: int64

In [19]:
# Вот так это работает
train['time1'].apply(lambda time: time.year).head() 

session_id
1    2014
2    2014
3    2013
4    2014
5    2014
Name: time1, dtype: int64

In [20]:
# А теперь пробуем, соединяем с месяцем (так, чтобы бло везде число одного порядка, а не 20221 и 202212)
train['time1'].apply(lambda time: time.year * 100 + time.month).head() 

session_id
1    201402
2    201402
3    201312
4    201403
5    201402
Name: time1, dtype: int64

**Создадим Df новых признаков, одинаково заполняем трэйн и тест выборки.**

In [183]:
train.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0


In [184]:
new_feat_train = pd.DataFrame(index=train.index)
new_feat_test = pd.DataFrame(index=test.index)

In [185]:
# Заносим
new_feat_train['year_month'] = train['time1'].apply(lambda tm: int(tm.year * 100 + tm.month))
new_feat_test['year_month'] = test['time1'].apply(lambda tm: int(tm.year * 100 + tm.month))

In [186]:
new_feat_train.head()

,year_month
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


**Если его добавить, то обязательно стандартизировать!!!!!!!!**

**Стандарт скалер из каждого признака вычтет среднее по столбцу и поделит на стандартное отклонение**

In [187]:
# Много-много возни с решейпом! Будет кидать ошибку и просьбу отрешейпить, когда передается 1 признак!
scaler = StandardScaler()
scaler.fit(new_feat_train['year_month'].values.reshape(-1, 1))

StandardScaler()

In [188]:
new_feat_train['year_month_scaled'] = scaler.transform(new_feat_train['year_month'].values.reshape(-1, 1))

In [189]:
# Все получилось!!!
new_feat_train

,year_month,year_month_scaled
session_id,,
21669,201301,-1.744405
54843,201301,-1.744405
77292,201301,-1.744405
114021,201301,-1.744405
146670,201301,-1.744405
...,...,...
12224,201404,0.681626
164438,201404,0.681626
12221,201404,0.681626


In [190]:
# Повторим на тестовой выборке, но !!! тестовая выборка не участвует в фите, чтобы не собрать информацию о ней
new_feat_test['year_month_scaled'] = scaler.transform(new_feat_test['year_month'].values.reshape(-1, 1))

In [191]:
ym = new_feat_train

In [192]:
ym['tar'] = train['target'] # Добавили Алису

In [193]:
ym

,year_month,year_month_scaled,tar
session_id,,,
21669,201301,-1.744405,0
54843,201301,-1.744405,0
77292,201301,-1.744405,0
114021,201301,-1.744405,0
146670,201301,-1.744405,0
...,...,...,...
12224,201404,0.681626,0
164438,201404,0.681626,0
12221,201404,0.681626,0


In [219]:
ym[ym.tar == 1]['year_month'].value_counts() # Посмотрели в какие мес Алиса активна

201311    446
201402    410
201403    400
201309    377
201404    302
201312    134
201401    129
201302     61
201304     38
Name: year_month, dtype: int64

In [217]:
ym['year_month'].value_counts() # Посмотрели в какие мес какая активность

201403    56250
201402    55563
201401    35869
201311    34493
201404    31204
201312    16552
201304     3997
201302     3929
201309     3464
201303     3337
201305     3229
201310     2788
201306     2094
201301      363
201308      290
201307      139
Name: year_month, dtype: int64

In [195]:
X_train_sparse.shape

(253561, 48371)

In [196]:
new_feat_train.shape

(253561, 3)

In [197]:
# Так как начальная матрица разреженая, то просто объединить не получится, надо использовать hstack
# На hstack тоже ругается, хочет один вектор-столбец!
X_train_new = csr_matrix(hstack([X_train_sparse, new_feat_train['year_month_scaled'].values.reshape(-1, 1)]))

In [198]:
# Видим, что размерность увеличилась на 1 столбец
X_train_new.shape

(253561, 48372)

In [199]:
# Получается почемуто меньше, чем в kaggle и меньше чем в лекции и меньше чем в первй раз)))))))))))
LogReg_roc_auc1(X_train_new, y_train, m_iter=500)

0.8237135489004861

## Добавим категорию начала первой сессии по времени суток

In [201]:
train.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0


In [202]:
train['time1'][1].hour

10

In [203]:
time_start = pd.DataFrame(index=train.index)

In [204]:
time_start['t_s'] = train['time1']

In [205]:
time_start['start'] = time_start['t_s'].apply(lambda ts: 0 if 9<=ts.hour<21 else 1)

In [206]:
# Признак начала сессии
time_start['start'].value_counts()

0    223340
1     30221
Name: start, dtype: int64

In [207]:
time_end = pd.DataFrame(index=train.index)
time_end['t_e'] = train['time10']
time_end['end'] = time_end['t_e'].apply(lambda te: 0 if 9<= te.hour < 21 else 1)
time_end['end'].value_counts()

0    204701
1     48860
Name: end, dtype: int64

In [208]:
time_end['start'] = time_start['start']

In [209]:
time_end['s_t'] = time_start['t_s']

### Нашел как вычислить быстро timedelta в секундах

In [210]:
type((time_end['t_e'][10] - time_end['s_t'][10]).total_seconds())

float

In [211]:
time_end['delta_st_end'] = time_end['t_e'] - time_end['s_t']

In [212]:
time_end['delta_st_end'] = time_end['delta_st_end'].apply(lambda td: td.total_seconds())

In [213]:
time_st_end_delta = time_end.drop('t_e', axis=1).drop('s_t', axis=1)

In [214]:
# Не понятно, надо ли НАНы в нули переделывать?
time_st_end_delta.fillna(0)

,end,start,delta_st_end
session_id,,,
21669,1,1,0.0
54843,1,1,0.0
77292,1,1,4.0
114021,1,1,3.0
146670,1,1,2.0
...,...,...,...
12224,1,1,12.0
164438,1,1,178.0
12221,1,1,28.0


In [216]:
time_st_end_delta['delta_st_end'].median() # медиана времени всех сессий

27.0

In [87]:
time_st_end_delta = time_st_end_delta.fillna(0)

In [88]:
# Сделаем две категории - вся сессия меньше 30 сек и вся сессия больше 30 сек
time_st_end_delta['delta_st_end'] = time_st_end_delta['delta_st_end'].apply(lambda tdse: 0 if tdse < 10 else 1)

In [89]:
time_st_end_delta['delta_st_end'].value_counts()

1    161491
0     92070
Name: delta_st_end, dtype: int64

In [90]:
X_train_new2 = csr_matrix(hstack([X_train_new, time_st_end_delta]))

In [100]:
X_train_new2

<253561x48375 sparse matrix of type '<class 'numpy.float64'>'
	with 1923809 stored elements in Compressed Sparse Row format>

In [97]:
X_train_new2.shape

(253561, 48375)

In [92]:
%%time
LogReg_roc_auc(X_train_new2, y_train)

CPU times: user 158 ms, sys: 16.6 ms, total: 174 ms
Wall time: 8.52 s


0.9731855663832912

In [93]:
%%time
# Вот это явное улучшение!!!!!!!!! При менее 30 сек на 0.15 от 0.8237 
# И еще лучше при менее 10 сек!
LogReg_roc_auc1(X_train_new2, y_train, m_iter=500)

CPU times: user 190 ms, sys: 12.2 ms, total: 202 ms
Wall time: 2.29 s


0.838500905479318

## Еще несколько признаков, посмотрим время каждой сессии

In [94]:
train_tm = train[times]

In [95]:
# Интересно, ругается, но дельту посчитал и правильно названия проставил
for i in range(0, 11):
    train_tm['delta%s' % i] = train_tm.iloc[:, i + 1] - train_tm.iloc[:, i]

/tmp/ipykernel_16475/4143465377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_tm['delta%s' % i] = train_tm.iloc[:, i + 1] - train_tm.iloc[:, i]
/tmp/ipykernel_16475/4143465377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_tm['delta%s' % i] = train_tm.iloc[:, i + 1] - train_tm.iloc[:, i]
/tmp/ipykernel_16475/4143465377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

TypeError: cannot subtract a datelike from a TimedeltaArray

In [102]:
train_tm['target'] = train['target']

/tmp/ipykernel_16475/1366894080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_tm['target'] = train['target']


In [111]:
train_tm[train_tm.target == 0]['delta0'].mean()

Timedelta('0 days 00:00:19.317000532')

In [106]:
train_tm

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,delta0,delta1,delta2,delta3,delta4,delta5,delta6,delta7,delta8,target
session_id,,,,,,,,,,,,,,,,,,,,
21669,2013-01-12 08:05:57,2013-01-12 08:05:57,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,0 days 00:00:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,0
54843,2013-01-12 08:37:23,2013-01-12 08:37:23,2013-01-12 09:07:07,2013-01-12 09:07:09,NaT,NaT,NaT,NaT,NaT,NaT,0 days 00:00:00,0 days 00:29:44,0 days 00:00:02,NaT,NaT,NaT,NaT,NaT,NaT,0
77292,2013-01-12 08:50:13,2013-01-12 08:50:14,2013-01-12 08:50:15,2013-01-12 08:50:15,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:17,2013-01-12 08:50:17,0 days 00:00:01,0 days 00:00:01,0 days 00:00:00,0 days 00:00:01,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:01,0 days 00:00:00,0
114021,2013-01-12 08:50:17,2013-01-12 08:50:17,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:20,0 days 00:00:00,0 days 00:00:01,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:01,0 days 00:00:00,0 days 00:00:00,0 days 00:00:01,0
146670,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:22,2013-01-12 08:50:22,2013-01-12 08:50:22,0 days 00:00:00,0 days 00:00:00,0 days 00:00:01,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:01,0 days 00:00:00,0 days 00:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12224,2014-04-30 23:33:48,2014-04-30 23:33:49,2014-04-30 23:33:52,2014-04-30 23:33:52,2014-04-30 23:33:53,2014-04-30 23:33:53,2014-04-30 23:33:54,2014-04-30 23:33:54,2014-04-30 23:33:57,2014-04-30 23:34:00,0 days 00:00:01,0 days 00:00:03,0 days 00:00:00,0 days 00:00:01,0 days 00:00:00,0 days 00:00:01,0 days 00:00:00,0 days 00:00:03,0 days 00:00:03,0
164438,2014-04-30 23:34:15,2014-04-30 23:34:16,2014-04-30 23:34:17,2014-04-30 23:34:18,2014-04-30 23:34:18,2014-04-30 23:35:16,2014-04-30 23:35:29,2014-04-30 23:36:12,2014-04-30 23:36:42,2014-04-30 23:37:13,0 days 00:00:01,0 days 00:00:01,0 days 00:00:01,0 days 00:00:00,0 days 00:00:58,0 days 00:00:13,0 days 00:00:43,0 days 00:00:30,0 days 00:00:31,0
12221,2014-04-30 23:38:08,2014-04-30 23:38:10,2014-04-30 23:38:13,2014-04-30 23:38:18,2014-04-30 23:38:22,2014-04-30 23:38:24,2014-04-30 23:38:35,2014-04-30 23:38:35,2014-04-30 23:38:35,2014-04-30 23:38:36,0 days 00:00:02,0 days 00:00:03,0 days 00:00:05,0 days 00:00:04,0 days 00:00:02,0 days 00:00:11,0 days 00:00:00,0 days 00:00:00,0 days 00:00:01,0


In [ ]:
deltas = ['delta%s' % i for i in range(0, 9)]

In [ ]:
deltas

In [ ]:
train_delta = train_tm[deltas]

In [ ]:
# Опять ругается, но сделал же!
train_delta['delta0'] = train_delta['delta0'].apply(lambda t: t.total_seconds())

In [ ]:
train_delta

In [ ]:
train_delta['delta1'] = train_delta['delta1'].apply(lambda t: t.total_seconds())
train_delta['delta2'] = train_delta['delta2'].apply(lambda t: t.total_seconds())
train_delta['delta3'] = train_delta['delta3'].apply(lambda t: t.total_seconds())
train_delta['delta4'] = train_delta['delta4'].apply(lambda t: t.total_seconds())
train_delta['delta5'] = train_delta['delta5'].apply(lambda t: t.total_seconds())
train_delta['delta6'] = train_delta['delta6'].apply(lambda t: t.total_seconds())
train_delta['delta7'] = train_delta['delta7'].apply(lambda t: t.total_seconds())
train_delta['delta8'] = train_delta['delta8'].apply(lambda t: t.total_seconds())

In [ ]:
train_delta = train_delta.fillna(0)

In [ ]:
# ВОТ ЕЩЕ 8 КОЛОНОК ПРИЗНАКОВ!!!
train_delta

In [ ]:
# Также забацаем категории
for i in range(0, 9):
    train_delta.iloc[:, i] = train_delta.iloc[:, i].apply(lambda td: 0 if td < 30 else 1)

In [ ]:
train_delta['delta6'].value_counts()

In [ ]:
X_train_new3 = csr_matrix(hstack([X_train_new2, train_delta]))

In [ ]:
X_train_new3.shape

In [ ]:
%%time
LogReg_roc_auc1(X_train_new3, y_train, m_iter=500)

In [ ]:
X_train_new2.shape, X_test_sparse.shape 

In [ ]:
X_test_sparse

In [ ]:
%%time
# Просто для оценки времени работы ф-ии, при сложных DFтак делать не надо, долго придется ждать
l_r = LogisticRegression(max_iter=1000, n_jobs=-1, random_state=17)
l_r_fit = l_r.fit(X_train_new2, y_train)

In [ ]:
l_r_predict_pr = l_r_fit.predict_proba(X_test_sparse)